In [218]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from itertools import combinations
import string
from collections import Counter, defaultdict
import re

In [219]:
path_connected_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\"
path_temp_data = "C:\\Users\\tykun\\OneDrive\\Documents\\SchoolDocs\\VSCodeProjects\\connectedData\\temporaryData\\"
year = "2009"

full_path = f"{path_connected_data}{year}_split_positions.csv"
dean_path = f"{path_temp_data}{year}_deans.csv"
grouped_subinstitution_path = f"{path_temp_data}{year}_grouped.csv"

pd.options.display.max_rows = 400

In [220]:
POSITION_BANK = ["President", "Chancellor", "Provost", "Director", "Dean", "Controller", "Trustee", "Member", "Regent", "Chairman", "Overseer", "Assistant", "Librarian", "Secretary", "Chaplain", "Minister", "Treasurer", "Senior Counsel", "General Counsel", "Legal Counsel", "University Counsel", "College Counsel", "Special Counsel", "Corporation Counsel", "Corporate Counsel", "Officer", "Chief", "Professor", "Commissioner", "Fellow", "Chairperson", "Manager", "Clergy", "Coordinator", "Auditor", "Governor", "Representative", "Stockbroker", "Advisor", "Commandant", "Rector", "Attorney", "Curator", "Clerk", "Department Head", "Pastor", "Head", "Comptroller", "Deputy", "Inspector General", "Instructor", "Registrar", "Ombuds", "Administrator", "Liaison", "Administrative Associate", "Webmaster", "Specialist", "University Planner", "Architect"]
#for classifying position as dean, administration
DEAN_WORDS = ["summer", "student", "faculty", "academic service", "academics", "academic program", "admissions", "admission", "enrollment", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", "university librar",
                       "internation affair", "special program", "continuing education", "external relation", "development", "services"]

#for subinstituion
ADMINISTRATION_WORDS = ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", "university librar"]

In [221]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics"],
    "Applied Science": ["applied science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childhood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies", "global management"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture "],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions"],
    "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
                       "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
                       "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
                       "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    "Satellite Campus": ["campus"],
    "Foundation": ["foundation"],
    "Boards": ["board of", "regents", "trustees", "council of"],
    "Throwaway": ["placeholder value"],
    "Hospital": ["hospital"],
    "University College": ["university college"],
    "Dean of the College": ["dean of the college", "dean, undergraduate college", "dean of the university"]
}

In [222]:
#slightly altered from the vp file
VP_KEYWORDS = {
    # "Head VP": ["head vice president"],
    # "SubInstitution": ["arts and science", "the college", "the arts", "interdisciplinary studies", "life sciences", "agriculture", "agricultural", "liberal arts", "engineering", "law center"],
    "Generic": [],
    "Foundation": ["foundation"],


    "Administration": ["vice president, administration", "and administration", "administrative", "audit and compliance", "legal affairs", "legal services", "institutional affairs", "university affairs", "vice president, operations", 
                       "employment service", "vice president, quality", "management service", "university compliance" "university operation", "institutional services",
                       "of the corporation", "special program", "university governance", "materials management", "executive affair"," university operation", "vice president. administration", "administration",
                       "resources"],
    "Board VP": ["of the board", "board of", "board operation"],
    "Campus Operations": ["sustainability", "facilities", "campus operations", "infrastructure", "environmental", "security", "campus police", "emergency management", "safety", 
                          "campus services", "university services", "facility operation", "university architect", "land, buildings and real estate", "auxiliary", "capital construction",
                          "property management", "plant operations", "construction", "campus residence", "physical resources", "university compliance", "physical plant", "campus involvement"],
    # "Health": ["health", "medical", "medicine", "ambulatory", "clinical service", "nursing", "nurse", "surgical", "rehabilitation", "hospital", "cancer", "patient care", "diagnostic service", "physicians",
    #            "clinical", "care contracting", "outpatient", "patient service"],


    "Academic Affairs": ["undergraduate education", "curriculum", "provost" "undergraduate studies", "graduate studies", "instruction", "academic program", "academic development",
                         "education affair", "undergraduate affair", "academie affairs", "educational and", "teacher education", "experiential education", "academic affair", "academic administration",
                         "undergraduate initiative", "cooperative education", "school program", "interdisciplinary education", "educational outreach", "educational,", "undergraduate studies", "educational affair",
                         "academic service", "academic afair", "educational resource", "academic enrichment", "academic resource", "university studies", "vice president, academics", "academic operation",
                         "educational development", "academic issues", "academic planning", "academic department"],
    "Faculty Affairs": ["faculty", "staff relations", "faculties"],
    "Research": ["innovation", "research", "grants", "publication", "biotechnology", "scientific", "sponsored programs"],
    

    "Student Affairs": ["student", "student life", "student service", "support", "wellbeing", "well-being", "campus life", "housing", "counseling", "student affairs", "student initiative", "student service", 
                        "student success", "student activities", "student development", "students", "university life", "student learning", "residence life", "student and community",
                        "career service", "student judicial service", "advisement", "student access", "study abroad", "dining program", "academic life", "freshmen", "freshman"],
    "Enrollment": ["enrollment", "financial aid", "retention", "registrar", "admission", "enrolment"],
    

    "Information and Technology": ["technology", "information", "librar", "data management", "computer", "computing", "network service", "internet service", "electronic marketing", "converging technologies",
                    "technical service", "vice president, systems", "enterprise system", "records", "system administration"],


    "DEI": ["diversity", "equity", "inclusion", "affirmative action", "dei", "multicultural affairs", "equal opportunity", "minority affair", "intercultural affair"],
    "External Relations": [ "external", "university relation", "college relation", "public relation", "communications", "public affair", "media", "outreach", "partnership", "global",
                            "international", "study abroad", "federal relations", "public events", "institutional relation",
                           "government", "state relations", "community engagement", "community affair", "civic engagement", "community relation", "corporate relation", 
                           "foundation relation", "legislative", "policy affairs", "defense relation", "constituent relations", "regional affair", "system relation", "regional operation",
                           "federal program", "agency relation", "regional engagement", "communication", "news services", "federal affair", "commonwealth relation", "state affair", "institute relation", "community services",
                           "intercampus", "urban affairs", "public service", "school relation", "public policy", "district of columbia affair", "d.c. affair", "vice president, engagement", "vice president, policy", "institute affair",
                           "community development"],


    "Planning": ["planning", "assessment", "evaluation", "planner", "special projects", "strategy and measurement", "institutional effectiveness", "presidential initiative", "capital improvement", "strategy and policy",
                 "university projects", "institutional analysis", "commercialization strategies", "procurement", "strategic affair", "university initiative", "program development", "strategic initiative", "strategic development",
                 "strategic and", "project", "program"],
    "Finance": ["budget", "finance", "financial", "accounting", "marketing", "endowment",  "investment", "business", "fiscal", "and resource management", "institutional resources", "treasury management", "university resources", "audit", ", resource management",
                "commerce", "asset management"],
    "Advancement": ["fundraise", "fundraising", "alumni", "donation", "parent relation", "annual giving", "special events", "principal gifts", "major gift", "campaigns", 
                    "planned giving", "leadership gifts", "gift program", "trusts and estate", "giving", "philanthropy", "constituent program", "institutional advancement", "vice president, advancement", "vice president, development",
                    "university advancement", "university development", "vice president. development", "vice president. advancement", "economic advancement", "economic development",
                    "college advancement", "institute advancement", "and development", "and advancement", "resource development", "institutional advancement", "sponsored funds", "college development", "wide advancement", "institution advancement",
                    "regional development", "development", "advancement"],
    "Human Resources": ["hr", "human resource", "personnel", "recruitment", "employee relations", "union", "collective bargaining", "labor", "human relations", "human capital", "personnel", "human and", "professional development", "human service", "hiring",
                        "workforce"],
    

    "Religion": ["ministry", "mission", "ministries", "church", "religious", "religion", "spiritual", "christian development"],
    "Satellite Campus": ["satellite campus"],
    "Athletics": ["athletic", "sports"],
    # "Agriculture": ["agriculture", "agricultural"],
    # "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning",
    #                        "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "extended program", "distance learning", "continuous education",
    #                        "online program", "online school", "online college", "online university", "distance education", "extended service"],
    "Graduate": ["graduate program", "graduate school", "graduate and", "advanced study", "graduate education"],

    "Other": []
}

In [223]:
admin_types = ["Administration", "Throwaway", "Generic", "Board VP", "Campus Operations", "Academic Affairs", "Faculty Affairs", "Student Affairs", "Enrollment", "Information and Technology", "External Relations", "Planning", "Finance", "Advancement", "Human Resources", "Religion", "Athletics"]

In [224]:
#Put every column from the xlsx into a df
def parse_columns(path, size):
    df = pd.read_csv(path)
    all_columns = df.iloc[:, :size]
    return all_columns


#Retrieve specific column by index
def retrieve_specific_column(df, index):
    return df.iloc[:, index]

#Extract the names of all the institutions for validation
def extract_institutions(df):
    institution_list = []
    for index, row in df.iterrows():
        if row["Institution"] not in institution_list:
            institution_list.append(row.iloc[2])
    return institution_list


def extract_first_member(df):
    first_member_df = []
    previous_institution = None
    for index, row in df.iterrows():
        current_institution = row["Institution"]
        if previous_institution is None or current_institution != previous_institution:
            first_member_df.append(row)
        previous_institution = current_institution
    return pd.DataFrame(first_member_df)

#extract first person from each institution
def extract_first_member_exclude_string(df, string):
    string = string.lower()
    first_member_df = []
    previous_institution = None
    for index, row in df.iterrows():
        current_institution = row["Institution"]
        # Check if the value at index 2 has changed, including handling NaN for the first row
        if previous_institution is None or current_institution != previous_institution:
            if string not in row.iloc[1].lower():
                first_member_df.append(row)
        previous_institution = current_institution
    return pd.DataFrame(first_member_df)

#Replace all the entries of a column with a standardized value (for president)
def replace_values(df, string):
    df.iloc[:,1] = string
    return df 

#Find the institutions that were left out
def find_missing_institutions(institutions, df):
    lowercase_df = df.iloc[:,2].str.lower()
    missing_institutions = []
    for institution in institutions:
        if institution.lower() not in lowercase_df.values:
            missing_institutions.append(institution)
    return missing_institutions


def rename_previous_value(df):
    df.rename(columns={'Previous_Value': 'Fixed Position'}, inplace = True)
    df["Fixed Position"] = ""
    return pd.DataFrame(df)


#validate university list
def count_universities(full_df):
    institutions = full_df["Institution"].unique()
    return list(institutions)

In [225]:
#Deans
#more to add here, but probably better to finish subinstitutions problem before doing this.
def extract_deans(df):
    dean_df = []
    for index, row in df.iterrows():
        if ('dean' in row.iloc[1].lower()) and "assistant to the dean" not in row.iloc[1].lower():
            dean_df.append(row)
    dean_df = pd.DataFrame(dean_df)
    return dean_df

#remove parentheses at end of string *used for Dean (interim) or Dean (acting)
def remove_parentheses(string):
    index = string.find('(')
    if index != -1:
        string = string[:index]
    return string.title()

def assign_position_grouping(position):
    classification = ""
    for key, value in VP_KEYWORDS.items():
        value_upper = [s.title() for s in value]
        keyword_appears = any(s in position for s in value_upper)
        dean_comma = position.rfind("Dean,") + 5
        if key.title() == position[dean_comma:]:
            classification += key + ", "
        elif keyword_appears:
            classification += key + ", "
    return classification.strip(", ")

def assign_position_grouping_subinst(position):
    classification = ""
    for key, value in keywords.items():
        value_upper = [s.title() for s in value]
        keyword_appears = any(s in position for s in value_upper)
        dean_comma = position.rfind("Dean,") + 5
        if key.title() == position[dean_comma:]:
            classification += key + ", "
        elif keyword_appears:
            classification += key + ", "
    return classification.strip(", ")


def assign_deans_subinstitution(deans_df, subinstitution_df):
    deans_df["Fixed Position"] = ""
    subinstitution_dict = subinstitution_df.set_index(['Institution', 'SubInstitution'])['Category'].to_dict()
    for index, row in deans_df.iterrows():
        position = row["Position"].title()
        altered_position = remove_parentheses(position).strip()
        subinstitution = row["SubInstitution"]
        dean_index = altered_position.rfind("Dean") + 4
        assigned = False
        #no text after "dean" and subinstitution exists
        if dean_index == len(altered_position) and type(subinstitution) != float:
            key = (row["Institution"], row["SubInstitution"])
            deans_df.at[index, "Fixed Position"] = subinstitution_dict[key]
        #text after "dean" and subinstitution exists
        elif dean_index != len(altered_position) and type(subinstitution) != float:
            #use subinstitution classification for the position
            classification = assign_position_grouping(position)
            deans_df.at[index, "Fixed Position"] = classification
            print(position, "    ", classification)
        elif dean_index != len(altered_position) and type(subinstitution) == float:
            classification = assign_position_grouping(position)
            deans_df.at[index, "Fixed Position"] = classification
            assigned = True
    return deans_df

def assign_remaining(deans_df, subinstitution_df):
    subinstitution_dict = subinstitution_df.set_index(['Institution', 'SubInstitution'])['Category'].to_dict()
    for index, row in deans_df.iterrows():
        fixed = row["Fixed Position"]
        subinstitution = row["SubInstitution"]
        position = row["Position"].title()
        dean_index= position.rfind("Dean") + 4
        if fixed == "":
            classification = assign_position_grouping_subinst(position)
            print(classification)
            deans_df.at[index, "Fixed Position"] = classification
        if dean_index == len(position) and fixed == "" and type(subinstitution) == float:
            deans_df.at[index, "Fixed Position"] = "Dean of the College"
        if "Dean Of The College" in position:
            deans_df.at[index, "Fixed Position"] = "Dean of the College"
    return deans_df
            

def assign_none(deans_df):
    for index, row in deans_df.iterrows():
        fixed = row["Fixed Position"]
        if fixed == "":
            deans_df.at[index, "Fixed Position"] = "Administration"
    return deans_df


def assign_administration(deans_df):
    for index, row in deans_df.iterrows():
        fixed = row["Fixed Position"]
        admin = any(s in fixed for s in admin_types)
        if admin:
            deans_df.at[index, "Fixed Position"] = "Administration"
    return deans_df

In [226]:
full_df = pd.read_csv(full_path)
subinstitution_df = pd.read_csv(grouped_subinstitution_path)

In [227]:
raw_deans = extract_deans(full_df)
cleaned_deans = assign_deans_subinstitution(raw_deans, subinstitution_df)
cleaned_deans = assign_remaining(cleaned_deans, subinstitution_df)
cleaned_deans = assign_none(cleaned_deans)
cleaned_deans = assign_administration(cleaned_deans)
cleaned_deans = cleaned_deans.sort_values(by="Fixed Position")
cleaned_deans.to_csv(dean_path, index = False)
